In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import os, sys, shutil, importlib, glob
from tqdm import tqdm_notebook as tqdm

#config InlineBackend.figure_format = 'retina'

#plt.rcParams['figure.figsize'] = [6,4.5]
#plt.rcParams["savefig.dpi"] = 300

In [2]:
from celloracle import motif_analysis as ma
import celloracle as co
from celloracle.utility import save_as_pickled_object
co.check_python_requirements()

/bgfs/rgottschalk/Sonia/envs/celloracle/lib/python3.8/site-packages/loompy/bus_file.py:68: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def twobit_to_dna(twobit: int, size: int) -> str:
/bgfs/rgottschalk/Sonia/envs/celloracle/lib/python3.8/site-packages/loompy/bus_file.py:85: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def dna_to_twobit(dna: str) -> int:
/bgf

,package_name,installed_version,required_version,requirement_satisfied
0,numpy,1.24.3,auto,True
1,scipy,1.10.1,auto,True
2,cython,3.0.10,auto,True
3,numba,0.58.1,0.50.1,True
4,matplotlib,3.6.3,auto,True
5,seaborn,0.13.2,auto,True
6,scikit-learn,1.3.0,auto,True
7,h5py,3.11.0,3.1.0,True
8,pandas,1.5.3,1.0.3,True
9,velocyto,0.17.17,0.17,True


In [3]:
from gimmemotifs.scanner import Scanner


In [4]:
def format_tss_annotation(tss_annotated):
    peak_id_tss = ma.process_bed_file.df_to_list_peakstr(tss_annotated)
    tss_annotated = pd.DataFrame({"peak_id": peak_id_tss,
                              "gene_short_name": tss_annotated.gene_short_name.values})
    tss_annotated = tss_annotated.reset_index(drop=True)
    return tss_annotated

In [5]:
chrombpnet_bed_file = "/bgfs/rgottschalk/Sonia/chrombpnet_AM_GSE100738/prediction_average_sorted_filt04_d5_merge.bed"

In [6]:
chrombpnet_bed = ma.read_bed(chrombpnet_bed_file)
chrombpnet_bed.head()

,chrom,start,end,seqname
0,chr1,4785192,4785233,chr1_4785192_4785233
1,chr1,4785241,4785249,chr1_4785241_4785249
2,chr1,4785257,4785343,chr1_4785257_4785343
3,chr1,4785388,4785407,chr1_4785388_4785407
4,chr1,4785417,4785915,chr1_4785417_4785915


In [8]:
chrombpnet_peaks = ma.process_bed_file.df_to_list_peakstr(chrombpnet_bed)

In [9]:
chrombpnet_tss_annotated = ma.get_tss_info(peak_str_list=chrombpnet_peaks, ref_genome="mm10")
chrombpnet_tss_annotated = format_tss_annotation(chrombpnet_tss_annotated)
chrombpnet_tss_annotated.to_csv('chrombpnet_baseGRN_processed_peaks.csv')
chrombpnet_tss_annotated.head()

que bed peaks: 149628
tss peaks in que: 37056


,peak_id,gene_short_name
0,chr12_102757451_102758124,Ubr7
1,chr12_102758130_102758415,Ubr7
2,chr14_30951008_30951038,Nek4
3,chr14_30951046_30951873,Nek4
4,chr8_95331831_95332057,Usb1


In [11]:
from gimmemotifs.motif import default_motifs, MotifConfig, read_motifs
config = MotifConfig()
motif_dir = config.get_motif_dir()
path = os.path.join(motif_dir, "JASPAR2022_vertebrates.pfm")
motifs = read_motifs(path)
motifs[:10]

[MA0004.1_MA0004.1.Arnt_CACGTG,
 MA0006.1_MA0006.1.Ahr::Arnt_yGCGTG,
 MA0019.1_MA0019.1.Ddit3::Cebpa_nnrTGCAATmCC,
 MA0029.1_MA0029.1.Mecom_AAGAyAAGATAAnA,
 MA0030.1_MA0030.1.FOXF2_nnAnsGTAAACAAn,
 MA0031.1_MA0031.1.FOXD1_GTAAACAw,
 MA0040.1_MA0040.1.Foxq1_nATTGTTTATw,
 MA0051.1_MA0051.1.IRF2_sGAAAGyGAAAsCnwwnn,
 MA0059.1_MA0059.1.MAX::MYC_rAsCACGTGGT,
 MA0066.1_MA0066.1.PPARG_sTAGGTCACnGTGACCyAnT]

In [14]:
ref_genome = "mm10"
geno_dir = "/bgfs/rgottschalk/Sonia/chrombpnet_AM_GSE100738/data"
chrombpnet_peaks = ma.check_peak_format(chrombpnet_tss_annotated, ref_genome, genomes_dir=geno_dir)

Peaks before filtering:  37056
Peaks with invalid chr_name:  0
Peaks with invalid length:  11344
Peaks after filtering:  25712


In [8]:
genome_installation = ma.is_genome_installed(ref_genome=ref_genome, genomes_dir=geno_dir)
print(ref_genome, "installation: ", genome_installation)

mm10 installation:  True


In [9]:
#do this if you're getting an error for scan
#import genomepy
#genomepy.install_genome(name=ref_genome, provider='UCSC', genomes_dir=None)

In [19]:
tfi = ma.TFinfo(peak_data_frame=chrombpnet_tss_annotated, ref_genome=ref_genome, genomes_dir=geno_dir)
tfi.scan(background_length = 200, n_cpus = 2, fpr=0.02, motifs=None, verbose=True)
tfi.to_hdf5(file_path="chrombpnetAM.celloracle.tfinfo")

No motif data entered. Loading default motifs for your species ...
 Default motif for vertebrate: gimme.vertebrate.v5.0. 
 For more information, please see https://gimmemotifs.readthedocs.io/en/master/overview.html 

Initiating scanner... 



2024-09-16 16:14:14,313 - DEBUG - using background: genome mm10 with size 200


Calculating FPR-based threshold. This step may take substantial time when you load a new ref-genome. It will be done quicker on the second time. 

Motif scan started .. It may take long time.



Scanning:   0%|          | 0/35262 [00:00<?, ? sequences/s]

In [20]:
tfi.scanned_df.tail()

,seqname,motif_id,factors_direct,factors_indirect,score,pos,strand
4678070,chrY_90785157_90785196,GM.5.0.C2H2_ZF.0106,"Osr1, Osr2","OSR1, OSR2",7.203072,27,-1
4678071,chrY_90785157_90785196,GM.5.0.GATA.0024,,GATA3,5.345995,21,-1
4678072,chrY_90785157_90785196,GM.5.0.C2H2_ZF.0299,,OSR2,8.790806,27,-1
4678073,chrY_90785252_90785286,GM.5.0.Unknown.0072,,TRIM28,11.463750,7,-1
4678074,chrY_90785252_90785286,GM.5.0.Forkhead.0055,,"FOXD4L1, Foxd3, FOXS1, Foxf2, Foxe3, FOXD4L5, ...",9.795725,11,-1


In [21]:
tfi.reset_filtering()

# Do filtering
tfi.filter_motifs_by_score(threshold=10)

# Format post-filtering results.
tfi.make_TFinfo_dataframe_and_dictionary(verbose=True)
df = tfi.to_dataframe()
df.head()

Filtering finished: 4678075 -> 966724
1. Converting scanned results into one-hot encoded dataframe.


  0%|          | 0/15906 [00:00<?, ?it/s]

2. Converting results into dictionaries.


  0%|          | 0/10417 [00:00<?, ?it/s]

  0%|          | 0/1091 [00:00<?, ?it/s]

,peak_id,gene_short_name,9430076c15rik,Ac002126.6,Ac012531.1,Ac226150.2,Afp,Ahr,Ahrr,Aire,...,Znf784,Znf8,Znf816,Znf85,Zscan10,Zscan16,Zscan22,Zscan26,Zscan31,Zscan4
0,chr10_100487038_100487881,Tmtc3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,chr10_100588776_100589421,4930430F08Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,chr10_100589485_100589556,4930430F08Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,chr10_10472072_10472259,Adgb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,chr10_10472266_10472705,Adgb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
df.to_parquet("chrombpnet_base_GRN_dataframe.parquet")